In [32]:
import numpy as np
import pandas as pd

In [33]:
bikes = pd.read_csv("C:/Users/Laura/Documents/DSML/Project/bluebikes_2021.csv")


C:\Users\Laura\AppData\Local\Temp\ipykernel_14812\70620609.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  bikes = pd.read_csv("C:/Users/Laura/Documents/DSML/Project/bluebikes_2021.csv")


# Exploration

In [34]:
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2934388 entries, 0 to 2934387
Data columns (total 10 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 0         int64  
 1   start_time         object 
 2   end_time           object 
 3   start_station_id   int64  
 4   start_station_lat  float64
 5   start_station_lon  float64
 6   end_station_id     object 
 7   end_station_lat    float64
 8   end_station_lon    float64
 9   bike_id            int64  
dtypes: float64(4), int64(3), object(3)
memory usage: 223.9+ MB


In [35]:
# explain what we found in the data frame (e.g. inconsistencies with column types, start_time / end-time columns, etc.)


# Data cleaning

In [36]:
# explain how we are going to fix all the mistakes we found

In [37]:
# first we found that there are inconsistencies in the columns start_time and end_time:
# some of the entries don't follow the format year-month-day; hours:minutes:seconds
# to be able to change the format first we need the replace the slashes (/) with colons (:) of some values

def substitute_slashes(data, column):
    data[column] = data[column].str.replace('/', ':')
    return data

substitute_slashes(bikes, 'start_time')
substitute_slashes(bikes, 'end_time')

,Unnamed: 0,start_time,end_time,start_station_id,start_station_lat,start_station_lon,end_station_id,end_station_lat,end_station_lon,bike_id
0,0,2021-01-01 00:00:04,2021-01-01 00:15:19,91,42.366277,-71.091690,370,42.350961,-71.077828,5316
1,1,2021-01-01 00:00:21,2021-01-01 00:18:27,370,42.350961,-71.077828,169,42.378965,-71.068607,4917
2,2,2021-01-01 00:00:26,2021-01-01 00:16:12,46,42.343666,-71.085824,21,42.346520,-71.080658,2881
3,3,2021-01-01 00:00:30,2021-01-01 00:06:26,178,42.359573,-71.101295,107,42.362500,-71.088220,4792
4,4,2021-01-01 00:01:11,2021-01-01 00:09:43,386,42.368605,-71.099302,413,42.369553,-71.085790,6062
...,...,...,...,...,...,...,...,...,...,...
2934383,2934373,2021-12-31 23:58:38,2022-01-01 00:06:38,4,42.345392,-71.069616,415,42.349544,-71.072421,4895
2934384,2934374,2021-12-31 23:58:46,2022-01-01 00:11:24,77,42.386844,-71.098120,96,42.373379,-71.111075,3665
2934385,2934375,2021-12-31 23:58:49,2022-01-01 00:03:30,338,42.348359,-71.139972,8,42.353334,-71.137313,6470
2934386,2934376,2021-12-31 23:59:50,2022-01-01 00:01:49,32,42.343691,-71.102353,19,42.347241,-71.105301,6484


In [38]:
# second, we transform the start_time and end_time columns to the proper format with the pd.to_datatime method

bikes[['start_time','end_time']] = bikes[['start_time','end_time']].apply(pd.to_datetime)

In [39]:
# another mistake is that the variable 'end_station_id' is an object, when it should be an integer like 'start_station_id'
# that is because some of the values in the column are also not in the proper format, instead of being whole numers
# some of them have quote symbols (") which makes the program transform the column into an object type
# so what we need to do is eliminate the quote marks of the numbers with the next function

def remove_quotes(data, column):
    data[column] = data[column].apply(lambda x: x.replace('"', '') if isinstance(x, str) else x)
    return data

bikes = remove_quotes(bikes, 'end_station_id')


In [40]:
# once all the values in the column have the same format, we can proceed to transform 
# the variable type of 'end_sation_id' to integer

def convert_column_to_integer(dataset, column_index):
    dataset[column_index] = pd.to_numeric(dataset[column_index], errors='coerce').astype('int64')
    return dataset

convert_column_to_integer(bikes, 'end_station_id')

,Unnamed: 0,start_time,end_time,start_station_id,start_station_lat,start_station_lon,end_station_id,end_station_lat,end_station_lon,bike_id
0,0,2021-01-01 00:00:04,2021-01-01 00:15:19,91,42.366277,-71.091690,370,42.350961,-71.077828,5316
1,1,2021-01-01 00:00:21,2021-01-01 00:18:27,370,42.350961,-71.077828,169,42.378965,-71.068607,4917
2,2,2021-01-01 00:00:26,2021-01-01 00:16:12,46,42.343666,-71.085824,21,42.346520,-71.080658,2881
3,3,2021-01-01 00:00:30,2021-01-01 00:06:26,178,42.359573,-71.101295,107,42.362500,-71.088220,4792
4,4,2021-01-01 00:01:11,2021-01-01 00:09:43,386,42.368605,-71.099302,413,42.369553,-71.085790,6062
...,...,...,...,...,...,...,...,...,...,...
2934383,2934373,2021-12-31 23:58:38,2022-01-01 00:06:38,4,42.345392,-71.069616,415,42.349544,-71.072421,4895
2934384,2934374,2021-12-31 23:58:46,2022-01-01 00:11:24,77,42.386844,-71.098120,96,42.373379,-71.111075,3665
2934385,2934375,2021-12-31 23:58:49,2022-01-01 00:03:30,338,42.348359,-71.139972,8,42.353334,-71.137313,6470
2934386,2934376,2021-12-31 23:59:50,2022-01-01 00:01:49,32,42.343691,-71.102353,19,42.347241,-71.105301,6484


In [41]:
# we can use the info() function to confirm that the type of the variable 'end_station_id' has been changed

bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2934388 entries, 0 to 2934387
Data columns (total 10 columns):
 #   Column             Dtype         
---  ------             -----         
 0   Unnamed: 0         int64         
 1   start_time         datetime64[ns]
 2   end_time           datetime64[ns]
 3   start_station_id   int64         
 4   start_station_lat  float64       
 5   start_station_lon  float64       
 6   end_station_id     int64         
 7   end_station_lat    float64       
 8   end_station_lon    float64       
 9   bike_id            int64         
dtypes: datetime64[ns](2), float64(4), int64(4)
memory usage: 223.9 MB


In [42]:
# note: i'm thinking that all of the 'id' variables shouldn't be integers 
# because we are not supposed to operate with those numbers but treat them as names, should be strings ???

In [43]:
bikes.sort_values(['bike_id', 'end_time'], inplace=True)
bikes

,Unnamed: 0,start_time,end_time,start_station_id,start_station_lat,start_station_lon,end_station_id,end_station_lat,end_station_lon,bike_id
356487,356485,2021-04-14 21:48:34,2021-04-14 22:03:09,171,42.374090,-71.069060,397,42.398361,-71.063738,218
364502,364500,2021-04-17 13:11:11,2021-04-17 13:44:59,397,42.398361,-71.063738,176,42.386748,-71.119019,218
366376,366374,2021-04-17 19:59:02,2021-04-17 20:22:45,176,42.386748,-71.119019,76,42.366426,-71.105495,218
366903,366901,2021-04-17 20:59:35,2021-04-17 21:23:46,76,42.366426,-71.105495,54,42.354979,-71.063348,218
367639,367637,2021-04-17 22:33:39,2021-04-17 22:59:36,54,42.354979,-71.063348,56,42.329843,-71.083866,218
...,...,...,...,...,...,...,...,...,...,...
2863170,2863160,2021-12-12 06:39:37,2021-12-12 06:53:46,74,42.373268,-71.118579,333,42.375002,-71.148716,7665
2869959,2869949,2021-12-13 14:29:21,2021-12-13 14:51:35,333,42.375002,-71.148716,76,42.366426,-71.105495,7665
2870114,2870104,2021-12-13 14:59:07,2021-12-13 15:09:55,76,42.366426,-71.105495,189,42.362428,-71.084955,7665
2882854,2882844,2021-12-15 13:53:04,2021-12-15 13:57:52,116,42.370803,-71.104412,68,42.365070,-71.103100,7665


In [44]:
bikes.sort_values(['bike_id', 'start_time'], inplace=True)

# Group the data by bike_id
grouped = bikes.groupby('bike_id')

# Calculate the idle time for each bike
bikes['idle_time'] = grouped['start_time'].shift(-1) - bikes['end_time']

# Set idle time to 0 for the last bike usage
bikes.loc[grouped['bike_id'].cumcount() == grouped['bike_id'].transform('size') - 1, 'idle_time'] = pd.Timedelta(0)

# Reset the dataframe index if needed
bikes.reset_index(drop=True, inplace=True)

In [45]:
bikes

,Unnamed: 0,start_time,end_time,start_station_id,start_station_lat,start_station_lon,end_station_id,end_station_lat,end_station_lon,bike_id,idle_time
0,356485,2021-04-14 21:48:34,2021-04-14 22:03:09,171,42.374090,-71.069060,397,42.398361,-71.063738,218,2 days 15:08:02
1,364500,2021-04-17 13:11:11,2021-04-17 13:44:59,397,42.398361,-71.063738,176,42.386748,-71.119019,218,0 days 06:14:03
2,366374,2021-04-17 19:59:02,2021-04-17 20:22:45,176,42.386748,-71.119019,76,42.366426,-71.105495,218,0 days 00:36:50
3,366901,2021-04-17 20:59:35,2021-04-17 21:23:46,76,42.366426,-71.105495,54,42.354979,-71.063348,218,0 days 01:09:53
4,367637,2021-04-17 22:33:39,2021-04-17 22:59:36,54,42.354979,-71.063348,56,42.329843,-71.083866,218,3 days 18:48:35
...,...,...,...,...,...,...,...,...,...,...,...
2934383,2863160,2021-12-12 06:39:37,2021-12-12 06:53:46,74,42.373268,-71.118579,333,42.375002,-71.148716,7665,1 days 07:35:35
2934384,2869949,2021-12-13 14:29:21,2021-12-13 14:51:35,333,42.375002,-71.148716,76,42.366426,-71.105495,7665,0 days 00:07:32
2934385,2870104,2021-12-13 14:59:07,2021-12-13 15:09:55,76,42.366426,-71.105495,189,42.362428,-71.084955,7665,1 days 22:43:09
2934386,2882844,2021-12-15 13:53:04,2021-12-15 13:57:52,116,42.370803,-71.104412,68,42.365070,-71.103100,7665,0 days 00:01:16


In [47]:
bikes.sort_values(['end_station_id'], inplace=True)
bikes

,Unnamed: 0,start_time,end_time,start_station_id,start_station_lat,start_station_lon,end_station_id,end_station_lat,end_station_lon,bike_id,idle_time
2159527,1354921,2021-08-01 12:38:53,2021-08-01 13:24:21,437,42.372077,-71.089954,1,42.387151,-71.075978,6044,3 days 00:15:31
1710203,1394447,2021-08-04 19:48:51,2021-08-21 23:01:42,222,42.343749,-71.062256,1,42.387151,-71.075978,5328,1 days 14:29:41
2645260,2796160,2021-12-01 20:35:58,2021-12-09 02:29:33,117,42.366088,-71.086336,1,42.387151,-71.075978,6770,0 days 18:28:23
370952,788494,2021-06-06 23:55:26,2021-06-29 15:32:25,403,42.339781,-71.121334,1,42.387151,-71.075978,2740,1 days 04:29:02
370841,367592,2021-04-17 22:26:25,2021-04-18 02:00:17,58,42.355536,-71.072869,1,42.387151,-71.075978,2740,18 days 19:08:15
...,...,...,...,...,...,...,...,...,...,...,...
1174852,2865736,2021-12-12 19:06:22,2021-12-12 19:23:23,60,42.360793,-71.071190,554,42.339202,-71.090511,4364,0 days 01:28:08
587095,2836447,2021-12-07 16:03:01,2021-12-07 16:25:41,225,42.371197,-71.097599,554,42.339202,-71.090511,3156,0 days 23:02:42
1786940,2871591,2021-12-13 19:40:10,2021-12-13 19:46:35,30,42.334629,-71.104079,554,42.339202,-71.090511,5459,0 days 00:36:28
1564566,2859778,2021-12-11 18:17:09,2021-12-11 18:24:05,364,42.338896,-71.081500,554,42.339202,-71.090511,5094,0 days 00:05:53


In [55]:
# Group the data by end_station_id and calculate the sum of idle time
station_idle_sum = bikes.groupby('end_station_id')['idle_time'].sum()

# Reset the index and rename the columns
station_idle_sum = station_idle_sum.reset_index()
station_idle_sum.columns = ['end_station_id', 'idle_time_sum']

station_idle_sum

,end_station_id,idle_time_sum
0,1,2319 days 09:23:43
1,3,2894 days 10:37:18
2,4,4750 days 06:43:39
3,5,1908 days 00:44:29
4,6,2990 days 23:36:04
...,...,...
422,548,6 days 00:50:59
423,549,1131 days 08:26:23
424,550,116 days 16:48:14
425,553,394 days 13:38:19


In [54]:
# Group the data by end_station_id and calculate the average idle time
station_idle_mean = bikes.groupby('end_station_id')['idle_time'].mean()

# Reset the index and rename the columns
station_idle_mean = station_idle_mean.reset_index()
station_idle_mean.columns = ['end_station_id', 'idle_time_mean']

station_idle_mean

,end_station_id,idle_time_mean
0,1,3 days 04:53:09.258287292
1,3,0 days 07:31:01.988745806
2,4,0 days 09:15:24.473773952
3,5,0 days 03:08:06.722511296
4,6,0 days 03:24:54.051569933
...,...,...
422,548,0 days 20:41:34.142857142
423,549,0 days 10:00:29.776262440
424,550,0 days 10:38:58
425,553,0 days 17:24:26.726102941
